# Chapter 1

### Pandas SQL

```
from pandasql import sqldf
import pandas as pd

# Create helper function for easier query execution
execute = lambda q: sqldf(q, globals())

# Load your CSV files into DataFrames
df1 = pd.read_csv("file1.csv")
df2 = pd.read_csv("file2.csv")

# Execute query with a join and store the result
query = """
    SELECT *
    FROM df1
    JOIN df2 ON df1.common_column = df2.common_column
"""
result_df = execute(query)

# Show results
result_df.head()
```

### Pandas sqlite

```
import sqlite3
import pandas as pd

def get_table_names(database_path='dataset/database.sqlite'):
    connection = sqlite3.connect(database_path)
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    result = connection.execute(query).fetchall()
    table_names = [row[0] for row in result]
    connection.close()
    return table_names

# Get and print all table names in the database
tables = get_table_names()
print("Tables in the database:", tables)

```

### CASE

```
SELECT
season,
ROUND(AVG(
    CASE         -- Start CASE
        WHEN hometeam_id = 8455 AND home_goal > away_goal THEN 1
        WHEN hometeam_id = 8455 AND home_goal < away_goal THEN 0
        ELSE NULL
    END          -- End CASE
        ), 2) AS pct_homewins,
FROM table_name
WHERE hometeam_id = 8455 OR awayteam_id = 8455  -- ADVISED : make sure to filter, else the new column will contain many NULLS with all data
GROUP BY season;
```